In [24]:
import os

In [ ]:
! pip install langchain_huggingface -q
! pip install unstructured -q
! pip install  unstructured[pdf] -q

In [26]:
import os
import logging
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.llms import HuggingFaceHub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# === CONFIG ===
DATA_PATH = "data"
DB_FAISS_PATH = "vectorstore/faiss_index"
HUGGINGFACE_REPO_ID = "google/flan-t5-base"
HF_TOKEN = os.getenv("HF_TOKEN")
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

In [ ]:

# === LOGGER ===
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [28]:
# === CORE FUNCTIONS ===
def get_embedding_model():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def load_pdfs():
    if not os.path.exists(DATA_PATH):
        raise FileNotFoundError("PDF data path not found.")
    return DirectoryLoader(DATA_PATH, glob="*.pdf").load()

def build_vectorstore():
    docs = load_pdfs()
    chunks = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
    ).split_documents(docs)

    db = FAISS.from_documents(chunks, get_embedding_model())
    os.makedirs(os.path.dirname(DB_FAISS_PATH), exist_ok=True)
    db.save_local(DB_FAISS_PATH)
    logger.info("Vectorstore saved successfully.")

def load_vectorstore():
    if not os.path.exists(DB_FAISS_PATH):
        raise FileNotFoundError("Vectorstore path not found.")
    return FAISS.load_local(DB_FAISS_PATH, get_embedding_model(), allow_dangerous_deserialization=True)

def create_qa_chain():
    retriever = load_vectorstore().as_retriever(search_kwargs={"k": 1})
    # llm = HuggingFaceHub(
    #     repo_id = HUGGINGFACE_REPO_ID,
    #     huggingfacehub_api_token = HF_TOKEN,
    #     model_kwargs = {"temperature": 0.5, "max_length": 512}
    # )
    
    llm = ChatGroq(
        model = ChatGroq(model = "llama-3.1-8b-instant")
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the medical question using only the given context in 2-3 sentences. If unsure, say 'I don't know'.\nContext: {context}"),
        ("human", "{input}")
    ])

    combine_docs_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(retriever, combine_docs_chain)


In [29]:
# === MAIN ===
if __name__ == "__main__":
    try:
        build_vectorstore()
        qa_chain = create_qa_chain()
        question = "What are the symptoms of diabetes?"
        answer = qa_chain.invoke({"input": question})
        print("\nAnswer:", answer.get("answer", "No answer returned."))
    except Exception as e:
        logger.error(f"Error: {e}")

ERROR:langchain_community.document_loaders.directory:Error loading file data\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf
ERROR:__main__:Error: partition_pdf() is not available because one or more dependencies are not installed. Use: pip install "unstructured[pdf]" (including quotes) to install the required dependencies
